In [1]:
import torch
from torchtext.datasets import AG_NEWS

news = AG_NEWS(split='train')
type(news)


torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe

In [9]:
train_iter = iter(AG_NEWS(split='train'))


In [10]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [12]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = [''])
vocab.set_default_index(vocab[''])

In [18]:
vocab(['here', 'is', 'an', 'example'])


[475, 21, 30, 5297]

In [21]:
EPOCHS = 10
LR = 5
BATCH_SIZE = 64
num_class = len(set([label for (label, text) in news]))
vocab_size = len(vocab)
emsize = 64

In [26]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [27]:
print(text_pipeline('here is an example'))
label_pipeline('10')

[475, 21, 30, 5297]


9

In [31]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse = True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [64]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    
    for idx, (label, text, offset) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offset)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for idx, (label, text, offset) in enumerate(dataloader):
            predicted_label = model(text, offset)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [35]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [59]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype = torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype = torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [60]:
train_iter, test_iter = AG_NEWS()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

In [61]:
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [62]:
next(iter(train_dataloader))

(tensor([3, 1, 0, 2, 1, 1, 1, 3, 1, 2, 3, 0, 2, 3, 0, 0, 0, 0, 0, 3, 2, 3, 1, 0,
         1, 3, 2, 1, 3, 2, 3, 2, 3, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 3, 2, 0, 1, 2,
         1, 3, 3, 1, 2, 0, 1, 2, 3, 0, 1, 0, 2, 2, 2, 0]),
 tensor([16892,   104,   363,  ...,    10,    57,     1]),
 tensor([   0,   23,   67,  140,  174,  206,  254,  296,  336,  363,  402,  426,
          458,  497,  534,  582,  631,  674,  733,  786,  827,  908,  945, 1010,
         1057, 1101, 1150, 1189, 1230, 1260, 1292, 1350, 1392, 1435, 1479, 1525,
         1570, 1605, 1659, 1692, 1740, 1784, 1829, 1879, 1932, 2010, 2046, 2083,
         2122, 2152, 2218, 2264, 2303, 2340, 2377, 2414, 2455, 2521, 2583, 2623,
         2657, 2694, 2745, 2782]))

In [66]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.1)

total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.920
| epoch   1 |  1000/ 1782 batches | accuracy    0.916
| epoch   1 |  1500/ 1782 batches | accuracy    0.920
-----------------------------------------------------------
| end of epoch   1 | time:  8.75s | valid accuracy    0.902 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.929
| epoch   2 |  1000/ 1782 batches | accuracy    0.926
| epoch   2 |  1500/ 1782 batches | accuracy    0.925
-----------------------------------------------------------
| end of epoch   2 | time:  8.96s | valid accuracy    0.897 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.944
| epoch   3 |  1000/ 1782 batches | accuracy    0.947
| epoch   3 |  1500/ 1782 batches | accuracy    0.946
-----------------------------------------------------------
| end of epoch   3 | time:  8.95s | valid accuracy    0.914 
-------------------------------

In [67]:
print(f'測試資料準確度: {evaluate(test_dataloader):.3f}')


測試資料準確度: 0.908


In [68]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text)).to(device)
        output = model(text, torch.tensor([0]).to(device))
        return output.argmax(1).item() + 1
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

print(ag_news_label[predict(ex_text_str, text_pipeline)])

Sports


In [69]:
my_test = open('./nlp_data/news.txt', encoding='utf8').read()
print(ag_news_label[predict(my_test, text_pipeline)])

Sports


## RNN model

In [70]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.rnn = nn.RNN(embed_dim, 32)
        self.fc = nn.Linear(32, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        rnn_out, h_out = self.rnn(embedded)
        return self.fc(rnn_out)

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)